In [3]:
from git import Repo# to clone the repository from github URLs
from langchain.text_splitter import Language #as we need to use python programming language here
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory #memory will be used as it will save the previous context memory
from langchain.chains import ConversationalRetrievalChain

In [4]:
%pwd

'd:\\ineuron\\Project_7_source_code_generation\\Source-Code-Analysis\\research'

In [5]:
!mkdir test_repo

In [6]:
repo_path = "test_repo/"

repo = Repo.clone_from("https://github.com/apoorvsinghnegi/Medical-Bot", to_path=repo_path) # to define our Github repository

In [7]:
#to load our source code from the github link that we have loaded or we hsve to clone from
loader = GenericLoader.from_filesystem(repo_path,
                                        glob = "**/*",
                                       suffixes=[".py"], #it will load all the .py or python files
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [8]:
documents = loader.load()

In [9]:
documents

[Document(page_content='', metadata={'source': 'test_repo\\ test.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_hugging_face_embeddings\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_openai import OpenAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\n\napp = Flask(__name__)\n\nload_dotenv()\n\nPINECONE_API_KEY=os.environ.get(\'PINECONE_API_KEY\')\nOPENAI_API_KEY=os.environ.get(\'OPENAI_API_KEY\')\n\nos.environ["PINECONE_API_KEY"] = PINECONE_API_KEY\nos.environ["OPENAI_API_KEY"] = OPENAI_API_KEY\n\nembeddings = download_hugging_face_embeddings()\n\n\nindex_name = "medicalbot"\n\n# Embed each chunk and upsert the embeddings into your Pinecone index

In [10]:
len(documents)

8

In [11]:
#context aware splittin - tag chunks with respect to the function by mentioning language parameter
#Even after chunking it would know from which function does this chunk belongs to.
#Context aware splitting only done for code data
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 500,
                                                             chunk_overlap = 20)

In [12]:
texts = documents_splitter.split_documents(documents)

In [13]:

len(texts)

15

In [16]:
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY=os.environ.get("OPENAI_API_KEY")

In [17]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [18]:
embeddings=OpenAIEmbeddings(disallowed_special=())

In [ ]:
#here we are intializing our chroma DB
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./db')

In [21]:
#to activate this vector DB
vectordb.persist()

NameError: name 'vectordb' is not defined

In [22]:
# llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI()

In [23]:
# to remember previous context we are using this memory
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [24]:
# conversational retrieval chain is a powerful framework that combines conversational memory with the ability to retrieve relevant information 
# from a knowledge base (like a document store or a vector database). 
# This setup is particularly useful when you want to maintain a context-aware conversation while retrieving external information based on 
# user queries.
#KEYWORD = 8 relevant answers
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":8}), memory=memory)

NameError: name 'vectordb' is not defined